In [1]:
import pandas as pd
import os

In [2]:
# This function creates the path to the main Excel file of this project
# Uses os library methods to ensure the file path works in both local and GitHub CI/CD environments
def create_file_path(file_relative_path):
    # Store the xlsx file dir into a var
    dir = '../data'
    
    # Iterate under dir data to have all files
    for filename in os.listdir(dir):
        # Condition to confirm which file is == year_make_model_df.xlsx
        if filename.endswith(file_relative_path):
            # Join the dir + filename to create the path to year_make_model_df.xlsx file  
            xlsx_file_fullpath = os.path.join(dir, filename)
            # Print the result
            print(xlsx_file_fullpath)
    return xlsx_file_fullpath

In [3]:
# Read excel 
df = pd.read_excel(create_file_path('year_make_model_df.xlsx'))

../data\year_make_model_df.xlsx


In [4]:
# Fuction to create df
def create_df(df, make):
    df_new = df[df['Make'] == make]
    return df_new

In [5]:
# Function to create list with unique items from given column
def data_from_column(df, column):
    # List to append the items
    data_column_list =[]

    # Iterate over given column
    for data in df[column]:
        # Append the items from the given column
        data_column_list.append(data)

    # Return a set with unique items from given column
    return set(data_column_list)

In [6]:
# Create a df to Lincoln make
df_mercury = create_df(df, 'Mercury')

In [7]:
# Check the string in the model column to confirm if changes are needed
data_from_column(df_mercury, 'Model')

{'Cougar',
 'Grand Marquis',
 'Marauder',
 'Mariner',
 'Mariner (Hybrid)',
 'Milan',
 'Milan (Hybrid)',
 'Montego',
 'Monterey',
 'Mountaineer (4dr)',
 'Mystique (V-6 only)',
 'Sable',
 'Sable (Duratec only)'}

In [8]:
# Check the string in the security column to confirm if changes are needed
data_from_column(df_mercury, 'Security')

{'Built February 1st 1998 or Earlier: PATS Type A (Stand Alone PATS Module)\nBuilt February 2nd 1998 or Later: PATS Type E (Powertrain Control Module)',
 'PATS Type A (Stand Alone PATS Module)',
 'PATS Type B (Stand Alone PATS Module)',
 'PATS Type C (Instrument Cluster)',
 'PATS Type E (Powertrain Control Module)'}

### Changes needed:
- There are 2 different models under 2 lines, they will be separated and the demiliter will be \n:
    * Built February 1st or earlier: PATS Type A (Stand Alone PATS Module)\nBuilt February 2nd or later: PATS Type E (Powertrain Control Module),
    * Built July 23rd 2000 or Earlier: PATS Type B (Stand Alone PATS Module)\nBuilt July 24th 2000 or Later: PATS Type E (Powertrain Control Module),

- The strings before (:) will moved from column security to column model as they are related to the models.
- This is the result on how it should be after updatings:  
Year: 2000  
Make: Mercury  
Model: M-XX (Built February 1st or earlier)   
Security: PATS Type G (Instrument Cluster)

In [9]:
# Fuction to create df based on a given string
def create_df_from_str(df, column, string):
    df_new = df[df[column] == string]
    return df_new

In [10]:
# Function to explode in different lines based on a given pattern
def explode_lines(df, column, pattern):
    # Make a copy of the original df
    df_exploded = df.copy()
    # Split the string in 2 based on the delimiter given and 
    # The result was given inside a list 
    df_exploded[column] = df_exploded[column].str.split(pattern)
    # Explode in 2 lines based on the list items quantity 
    # ['item1', 'item2'], exploded in 2 lines
    df_exploded = df_exploded.explode(column)
    # Return the df exploded
    return df_exploded

In [11]:
# Function to move the strings before char (:) from column security to column model
def move_string_to_column_model(df):
    # Function to explode in separate lines strings from security column
    df_exploded = explode_lines(df, 'Security', '\n')
    # Reset the index
    df_resetted_index = df_exploded.reset_index(drop=True)
    # Split the string in the security column using the (:) as the delimeter
    df_resetted_index['Security'] = df_resetted_index['Security'].str.split(":")

    # Loop to iterate under the df indexes
    for idx in df_resetted_index.index:
        # Method to move the manufactured date to the model column
        df_resetted_index['Model'][idx] = f'{df_resetted_index['Model'][idx]} ({df_resetted_index['Security'][idx][0]})'
        # Remove manufactured date from security column
        df_resetted_index['Security'][idx].pop(0)

    # Extracts the string from list under security column  
    df_security_list = df_resetted_index['Security'].str.join(' ')

    # Update the df
    df_resetted_index['Security'] = df_security_list

    # Return a df with the strings before (:) moved to column model
    return df_resetted_index

In [12]:
# Call the function to create a df with the string should be changed under column security
df_mercury_built_date = create_df_from_str(df_mercury, 'Security', 'Built February 1st 1998 or Earlier: PATS Type A (Stand Alone PATS Module)\nBuilt February 2nd 1998 or Later: PATS Type E (Powertrain Control Module)')

In [13]:
# Create a list with the indexes
mercury_built_date_indexes_list = list(df_mercury_built_date.index)

In [14]:
# Call the function to move the strings from column security to column model
df_mercury_built_date_moved_strings = move_string_to_column_model(df_mercury_built_date)

C:\Users\Weverson Barbieri\AppData\Local\Temp\ipykernel_3664\3062641589.py:13: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_resetted_index['Model'][idx] = f'{df_resetted_index['Model'][idx]} ({df_resetted_index['Security'][idx][0]})'
C:\

In [15]:
# Print the df after moving the strings
df_mercury_built_date_moved_strings

,Year,Make,Model,Security,ParameterReset
0,1998,Mercury,Mystique (V-6 only) (Built February 1st 1998 o...,PATS Type A (Stand Alone PATS Module),Parameter Reset Not Required
1,1998,Mercury,Mystique (V-6 only) (Built February 2nd 1998 o...,PATS Type E (Powertrain Control Module),Parameter Reset Not Required


In [16]:
# Concatenate the df with built date moved to the model column to the main df
df_mercury_concatenated = pd.concat([df_mercury, df_mercury_built_date_moved_strings])

In [17]:
# Drop the indexes with the built date at the security column
df_mercury_concatenated_dropped_indexes = df_mercury_concatenated.drop(index=mercury_built_date_indexes_list)

In [18]:
# Reset the index
df_mercury_reset_index = df_mercury_concatenated_dropped_indexes.reset_index(drop=True)

## Check the changes

In [19]:
# Column Model
data_from_column(df_mercury_reset_index, 'Model')

{'Cougar',
 'Grand Marquis',
 'Marauder',
 'Mariner',
 'Mariner (Hybrid)',
 'Milan',
 'Milan (Hybrid)',
 'Montego',
 'Monterey',
 'Mountaineer (4dr)',
 'Mystique (V-6 only)',
 'Mystique (V-6 only) (Built February 1st 1998 or Earlier)',
 'Mystique (V-6 only) (Built February 2nd 1998 or Later)',
 'Sable',
 'Sable (Duratec only)'}

In [20]:
# Column Security
data_from_column(df_mercury_reset_index, 'Security')

{' PATS Type A (Stand Alone PATS Module)',
 ' PATS Type E (Powertrain Control Module)',
 'PATS Type A (Stand Alone PATS Module)',
 'PATS Type B (Stand Alone PATS Module)',
 'PATS Type C (Instrument Cluster)',
 'PATS Type E (Powertrain Control Module)'}

### PATS Type and Anti-Theft Module Location separation:
PATS (Passive Anti-Theft System) Type and Anti-Theft Module Location are currently in the same column. However, the PATS Type defines which key learning procedure should be performed. To make it easier to indicate the correct procedure, it is necessary to separate this information. See below how it is currently and how it should look after the update.

- Before changes:  
Year: 2000  
Make: Mercury  
Model: M-XX (Built February 1st or earlier)   
Security: PATS Type G (Instrument Cluster)  

- After changes:  
Year: 2000  
Make: Mercury  
Model: M-XX  
PATS Type: PATS Type G  
Anti-Theft Module Location: Instrument Cluster

In [21]:
# This function will separate the columns as described above
def split_pats_type_from_module_location(df):

    # Split the security string considering the '(' as the delimiter
    df['Security'] = df['Security'].str.split('(')

    # Loop to iterate under the df indexes
    for idx in df.index:
        # Method to add a new column to the df, pulling the PATS Type from security column 
        df.at[idx, 'PATS Type'] = f'{df['Security'][idx][0]}'
        # Remove the PATS Type from the security column
        df['Security'][idx].pop(0)

    # Extracts the string from list under security column  
    df_anti_theft_module_location_list = df['Security'].str.join(' ')

    # Update the df
    df['Security'] = df_anti_theft_module_location_list

    # Remove char ')' from column security
    df['Security'] = df['Security'].str.replace(')', '', regex=False)

    # Rename column security to 'Anti-Theft Module Location'
    df = df.rename(columns={'Security': 'Anti-Theft Module Location'})

    # Reorder columns 
    df_updated_reordered_columns = df[['Year', 'Make', 'Model', 'Anti-Theft Module Location', 'PATS Type', 'ParameterReset']]

    # Return the df updated: columns for pats type and anti-theft module location separated
    return df_updated_reordered_columns

In [22]:
# Call the function to separate columns pats type, security, rename columns security and reorder the columns
df_mercury_reordering_column = split_pats_type_from_module_location(df_mercury_reset_index)

In [23]:
# Print the head to confirm the column reordering
df_mercury_reordering_column.head()

,Year,Make,Model,Anti-Theft Module Location,PATS Type,ParameterReset
0,1996,Mercury,Sable (Duratec only),Stand Alone PATS Module,PATS Type A,Parameter Reset Not Required
1,1997,Mercury,Sable (Duratec only),Stand Alone PATS Module,PATS Type A,Parameter Reset Not Required
2,1998,Mercury,Grand Marquis,Stand Alone PATS Module,PATS Type B,Parameter Reset Required
3,1998,Mercury,Mountaineer (4dr),Stand Alone PATS Module,PATS Type B,Parameter Reset Required
4,1998,Mercury,Sable (Duratec only),Stand Alone PATS Module,PATS Type B,Parameter Reset Required


### Check if the updates were done

In [24]:
# Column Anti-Theft Module Location
data_from_column(df_mercury_reordering_column, 'Anti-Theft Module Location')

{'Instrument Cluster', 'Powertrain Control Module', 'Stand Alone PATS Module'}

In [25]:
# Column PATS Type
data_from_column(df_mercury_reordering_column, 'PATS Type')

{' PATS Type A ',
 ' PATS Type E ',
 'PATS Type A ',
 'PATS Type B ',
 'PATS Type C ',
 'PATS Type E '}

In [26]:
# Check if column parameter reset needs to be change
data_from_column(df_mercury_reordering_column, 'ParameterReset')

{'Parameter Reset Not Required', 'Parameter Reset Required'}

### No changes needed in the ParameterReset column

In [27]:
# Export the to .csv file
df_mercury_reordering_column.to_csv('C:\\Language_Projects\\Language_Projects\\Python\\Flagship_1\\vehicle_security_system_data_cleaning\\data\\df_mercury.csv', index=False)